In [ ]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'\\CHCFPP01\Guest\MRajabi\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import scipy
from matplotlib.gridspec import GridSpec
from __future__ import print_function
from mailmerge import MailMerge
import shutil
import openpyxl
from scipy.optimize import curve_fit
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
Years = range(2004,2017)
# Roadway and Crash Data
Project_Dir = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\IL_Crash_Assignment'
SourceDir = Project_Dir + '\\CrashData_Original'
OutputDir = Project_Dir + '\\CrashData_Cleaned'
GDB = Project_Dir + '\\IL_Crash.gdb'
IRIS_Path = r'C:\Users\mr068144\Downloads\IL Data\IRIS\IL_IRIS.gdb'
Routes  = {year:IRIS_Path + '\\HWY{}_route'.format(year) for year in Years}
AttTabs  = {year:IRIS_Path + '\\HWY{}_table_HCurve'.format(year) for year in Years}
print(strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
#Read Crash Data
print(strftime("%Y-%m-%d %H:%M:%S"))
Crash_DF = pd.DataFrame()
for year in range(2005,2017):
    #df = pd.read_csv(r'C:\Users\mr068144\Downloads\IL Data\IL Crash Data'+'\\{}_Crash_Cleaned.csv'.format(year),index_col=0)
    df = pd.read_csv(OutputDir+'\\{}_Crash_Assigned.csv'.format(year),index_col=0)
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['YEAR'] = year
    #print(year,df.shape[0]-df.drop_duplicates().shape[0])
    Crash_DF = pd.concat([Crash_DF,df])
    print(year)
print(Crash_DF.shape)
print(strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
# Read Vehicle Data
print(strftime("%Y-%m-%d %H:%M:%S"))
Veh_DF = pd.DataFrame()
for year in range(2004,2017):
    df = pd.read_csv(OutputDir+'\\{}_Vehicle_Cleaned_EA.csv'.format(year),index_col=[0,1])
    df['YEAR'] = year
    Veh_DF = pd.concat([Veh_DF,df])
    print(year)
print(Veh_DF.shape)
print(strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
# Read Person Data
print(strftime("%Y-%m-%d %H:%M:%S"))
Occ_DF = pd.DataFrame()
for year in range(2004,2017):
    df = pd.read_csv(OutputDir+'\\{}_Person_Cleaned_EA.csv'.format(year),index_col=[0,1,2], encoding='latin-1',low_memory=False)
    df['YEAR'] = year
    Occ_DF = pd.concat([Occ_DF,df])
    print(year)
print(Occ_DF.shape)
print(strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
# Check EA Definitions
EA_DF = pd.DataFrame(index = ['RD','IM','UO','IN','SA','OD','YD','MC','HV','PD','PC','WZ','DF','TN'],columns=['K_Ref','K','A','B','C','O'])
EA_DF = EA_DF.fillna(0)
EA_DF['K_Ref'] = [2048,2088,1377,1178,1108,848,694,694,672,641,137,133,123,45]
for i in EA_DF.index:
    for c in ['K','A','B','C','O']:
        EA_DF.set_value(i,c,Occ_DF[(Occ_DF.YEAR.isin(range(2004,2017))) & (Occ_DF['EA_'+i]==1) & (Occ_DF.INJ_SEV=={'K':4,'A':3,'B':2,'C':1,'O':0}[c])].shape[0])
EA_DF

In [ ]:
EA_DF = pd.DataFrame(index=range(2004,2017),columns=pd.MultiIndex.from_product([['K','A','B','C'],['RD','IM','UO','IN','SA','OD','YD','MC','HV','PD','PC','WZ','DF','TN']],names=['Injuries','EA']))
EA_DF = EA_DF.fillna(0)
EA_DF = EA_DF.T
for i,r in EA_DF.iterrows():
    #print(i)
    df = pd.DataFrame(columns=['Numbers'],index=range(2004,2017))
    df.Numbers = pd.DataFrame(Occ_DF[(Occ_DF.INJ_SEV=={'K':4,'A':3,'B':2,'C':1}[i[0]]) & (Occ_DF['EA_'+i[1]]==1)].groupby('YEAR').size())
    EA_DF.loc[i]=list(df.Numbers)
EA_DF = EA_DF.fillna(0)
EA_DF = EA_DF.astype(int)
EA_DF

In [ ]:
EAs = [['RD','IM'],
['UO','IN','SA','OD'],
['YD','MC','HV','PD'],
['PC','WZ','DF','TN']]
for EA in EAs:
    plt.figure(figsize=(12, 8), dpi=300, facecolor='w', edgecolor='k')
    plt.plot(EA_DF.loc[('K')].T[EA])
    plt.xlim([2005,2017])
    plt.xticks(range(2005,2017),range(2005,2017))
    plt.legend(EA,loc = 'upper left')
    plt.grid()
    plt.show()